In [1]:
def dimension(Var,Equ,D):                                            #This function calculate the dimension of the affine space
    if Var==[]:                                                      #with variables in Var and equations in Equ via counting
        return D                                                     #points over F_2            
    var=list(Equ[0].variables());equ=[];equ.append(Equ[0]);i=1           
    
    while i<=len(Equ)-1:                                             #We split variables and equations into groups to make it 
        for j in range(len(var)):                                    #run faster
            if var[j] in Equ[i].variables():
                if Equ[i] in equ:
                    i+=1;break
                else:
                    var=list(set(var+list(Equ[i].variables())))
                    equ.append(Equ[i])
                    i=1
                    break
        else:
            i+=1
   
    
    for i in var:
        Var.remove(i)
    
    for i in equ:
        Equ.remove(i)
    X=PolynomialRing(GF(2),names=var)
    A=AffineSpace(X)
    Z=A.subscheme([equ[i] for i in range(len(equ))])
    from sage.schemes.affine.affine_rational_point import enum_affine_finite_field
    D+=log(len(enum_affine_finite_field(Z)),2)
    
    if Var==[]:
        return D
    return dimension(Var,Equ,D)

def relation(par,N):                                             #This funtion gives the information about the stratum
    a={(i,j,k,l,m,n):var("a_{}{}{}{}{}{}".format(i,j,k,l,m,n))   #associated to par with a bound N
       for i in [0..N] for j in [0..N] for k in [0..N] for l in [0..N] for m in [0..N] for n in [0..N]}
    
    n=0;Base=[];Comm=[];Var=[]                                   #Var contains variables a^*_*
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==1:
                    n+=1
                    Base.append((i,j,k))
                if par[i,j,k]==0:
                    if i>0 and par[i-1,j,k]==1:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Var.append(a[i,j,k,z,x,y])
                        continue
                    if j>0 and par[i,j-1,k]==1:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Var.append(a[i,j,k,z,x,y])
                        continue
                    if k>0 and par[i,j,k-1]==1:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Var.append(a[i,j,k,z,x,y])
                        continue

    
    M = MatrixSpace(SR,n)
    Tx = M(); Ty=M(); Tz=M()                                            #Tx,Ty,Tz are the formal multiplication matrices
    Cxz = M(); Cyz=M(); Cxy=M()
    
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==0:
                    continue
                if par[i,j,k]==1:
                    if par[i,j+1,k]==1:
                        Tx[Base.index((i,j+1,k)),Base.index((i,j,k))]=1
                    if par[i,j+1,k]==0:
                        for x in [j+1..N]:
                            for y in [0..N]:
                                if x==j+1 and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Tx[Base.index((z,x,y)),Base.index((i,j,k))]=a[i,j+1,k,z,x,y]     
                                        
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==0:
                    continue
                if par[i,j,k]==1:
                    if par[i,j,k+1]==1:
                        Ty[Base.index((i,j,k+1)),Base.index((i,j,k))]=1
                    if par[i,j,k+1]==0:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k+1:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Ty[Base.index((z,x,y)),Base.index((i,j,k))]=a[i,j,k+1,z,x,y]
                                        
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==0:
                    continue
                if par[i,j,k]==1:
                    if par[i+1,j,k]==1:
                        Tz[Base.index((i+1,j,k)),Base.index((i,j,k))]=1
                    if par[i+1,j,k]==0:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Tz[Base.index((z,x,y)),Base.index((i,j,k))]=a[i+1,j,k,z,x,y]  
    
    Cxz=Tx*Tz-Tz*Tx
    
    for i in [0..n-1]:
        for j in [0..n-1]:
            if Cxz[i,j]==0:
                continue
            Comm.append(Cxz[i,j])
    
    
    Cyz=Ty*Tz-Tz*Ty
    
    for i in [0..n-1]:
        for j in [0..n-1]:
            if Cyz[i,j]==0:
                continue
            Comm.append(Cyz[i,j])
            
    Cxy=Tx*Ty-Ty*Tx
    
    #for i in [0..n-1]:
        #for j in [0..n-1]:
            #if Cxy[i,j]==0:
                #continue
            #Comm.append(Cxy[i,j])
    
    for x in [0..N]:                                                       #If we have Tx,Tz and Ty,Tz commute with each other,
        for y in [0..N]:                                                   #we only need to have Tx,Ty commute for the lowest layer
            if par[0,x,y]==1:
                for i in [0..n-1]:
                    if Cxy[i,Base.index((0,x,y))]==0:
                        continue
                    Comm.append(Cxy[i,Base.index((0,x,y))])
    
    Comm=sorted(list(set(Comm)))                                           #Comm contains the defining equations of the stratum
    
    Var2=[]
    for i in [0..len(Comm)-1]:
        for j in [0..len(Comm[i].variables())-1]:
            Var2.append(Comm[i].variables()[j])
    
    Var2=sorted(list(set(Var2)))                                          #Var2 contains the variables that appear in the 
    print(Comm)                                                           #defining equations. We call them relevant variables
    print("relevant variables:", Var2)
    print("number of relevant variables:",len(Var2))
    print("number of irrelevant variables:", len(Var)-len(Var2))    
    
    D=0
    print("dimension:", len(Var)-len(Var2)+dimension(Var2,Comm,D))       #If the stratum is affine, this will give the 
                                                                          #dimension of the stratum
    
    

def transform(T,t):                                                       #This transform the partition into 
    for i in [0..len(T)-1]:                                               #monomial representations
        for j in [0..len(T[i])-1]:
            for k in [0..T[i][j]-1]:
                t[k,j,i]=1
                
t={(i,j,k):0 for i in [0..5] for j in [0..5] for k in [0..5]}             #This is an example for a partition T with a bound 5
                                                                          #If the stratum is not an affine space, the dimension
T=[[1,1],[1,1]]                                                           #may not be an integer
transform(T,t)
relation(t,5)



[a_100001 - a_110011, a_002010*a_100001 - a_101010, a_002011*a_100001 + a_100010 - a_101011, -a_002010*a_110011 + a_101010]
relevant variables: [a_101010, a_110011, a_100010, a_002011, a_100001, a_002010, a_101011]
number of relevant variables: 7
number of irrelevant variables: 1
dimension: 5


In [18]:


def dimension(Var,Equ,D):                                             #This is the same dimension function as above
    if Var==[]:
        return D
    var=list(Equ[0].variables());equ=[];equ.append(Equ[0]);i=1
    
    while i<=len(Equ)-1:
        for j in range(len(var)):
            if var[j] in Equ[i].variables():
                if Equ[i] in equ:
                    i+=1;break
                else:
                    var=list(set(var+list(Equ[i].variables())))
                    equ.append(Equ[i])
                    i=1
                    break
        else:
            i+=1 
   
    
    for i in var:
        Var.remove(i)
    
    for i in equ:
        Equ.remove(i)
    X=PolynomialRing(GF(2),names=var)
    A=AffineSpace(X)
    Z=A.subscheme([equ[i] for i in range(len(equ))])
    from sage.schemes.affine.affine_rational_point import enum_affine_finite_field
    D+=log(len(enum_affine_finite_field(Z)),2)
     
    return dimension(Var,Equ,D)


def Dimension(par,N,hd):                                          #This is almost the same as the 'relation' above. But this
    a={(i,j,k,l,m,n):var("a_{}{}{}{}{}{}".format(i,j,k,l,m,n))    #function only gives the dimension instead of relations
       for i in [0..N] for j in [0..N] for k in [0..N] for l in [0..N] for m in [0..N] for n in [0..N]}
    
    n=0;Base=[];Comm=[];Var=[]
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==1:
                    n+=1
                    Base.append((i,j,k))
                if par[i,j,k]==0:
                    if i>0 and par[i-1,j,k]==1:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Var.append(a[i,j,k,z,x,y])
                        continue
                    if j>0 and par[i,j-1,k]==1:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Var.append(a[i,j,k,z,x,y])
                        continue
                    if k>0 and par[i,j,k-1]==1:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Var.append(a[i,j,k,z,x,y])
                        continue

    
    M = MatrixSpace(SR,n)
    Tx = M(); Ty=M(); Tz=M()
    Cxz = M(); Cyz=M(); Cxy=M()
    
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==0:
                    continue
                if par[i,j,k]==1:
                    if par[i,j+1,k]==1:
                        Tx[Base.index((i,j+1,k)),Base.index((i,j,k))]=1
                    if par[i,j+1,k]==0:
                        for x in [j+1..N]:
                            for y in [0..N]:
                                if x==j+1 and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Tx[Base.index((z,x,y)),Base.index((i,j,k))]=a[i,j+1,k,z,x,y]     
                                        
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==0:
                    continue
                if par[i,j,k]==1:
                    if par[i,j,k+1]==1:
                        Ty[Base.index((i,j,k+1)),Base.index((i,j,k))]=1
                    if par[i,j,k+1]==0:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k+1:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Ty[Base.index((z,x,y)),Base.index((i,j,k))]=a[i,j,k+1,z,x,y]
                                        
    for i in [0..N]:
        for j in [0..N]:
            for k in [0..N]:
                if par[i,j,k]==0:
                    continue
                if par[i,j,k]==1:
                    if par[i+1,j,k]==1:
                        Tz[Base.index((i+1,j,k)),Base.index((i,j,k))]=1
                    if par[i+1,j,k]==0:
                        for x in [j..N]:
                            for y in [0..N]:
                                if x==j and y<=k:
                                    continue
                                for z in [0..N]:
                                    if par[z,x,y]==1:
                                        Tz[Base.index((z,x,y)),Base.index((i,j,k))]=a[i+1,j,k,z,x,y]  
    
    Cxz=Tx*Tz-Tz*Tx
    
    for i in [0..n-1]:
        for j in [0..n-1]:
            if Cxz[i,j]==0:
                continue
            Comm.append(Cxz[i,j])
    
    
    Cyz=Ty*Tz-Tz*Ty
    
    for i in [0..n-1]:
        for j in [0..n-1]:
            if Cyz[i,j]==0:
                continue
            Comm.append(Cyz[i,j])
            
    Cxy=Tx*Ty-Ty*Tx
    
    #for i in [0..n-1]:
        #for j in [0..n-1]:
            #if Cxy[i,j]==0:
                #continue
            #Comm.append(Cxy[i,j])
    
    for x in [0..N]:
        for y in [0..N]:
            if par[0,x,y]==1:
                for i in [0..n-1]:
                    if Cxy[i,Base.index((0,x,y))]==0:
                        continue
                    Comm.append(Cxy[i,Base.index((0,x,y))])
    
    Comm=sorted(list(set(Comm)))  
    
    Var2=[]
    for i in [0..len(Comm)-1]:
        for j in [0..len(Comm[i].variables())-1]:
            Var2.append(Comm[i].variables()[j])
    
    Var2=sorted(list(set(Var2))) 
       
    
    D=0                                                              
    return len(Var)-len(Var2)+dimension(Var2,Comm,D)

def transform(T,t):
    for i in [0..len(T)-1]:
        for j in [0..len(T[i])-1]:
            for k in [0..T[i][j]-1]:
                t[k,j,i]=1

def add(l,k):
    for i in l:
        yield i
    yield k
        
def P(n,xb,yb,l,L):                                                   #This function generates in L all 1-dimensional partitions
    if len(yb)==0:                                                    #of n that is bounded by the 1-dimensional partition yb  
        return
    for k in [1..min(xb,yb[0])]:
        if n==k:
            L.append(list(add(l,k)))
            return
        P(n-k,min(n-k,k),list(yb)[1:],list(add(l,k)),L)

def PP(n,YB,yb,L,temp,LL):                                            #This function generates in LL all 2-dimensional 
    for k in [1..YB]:                                                 #partitions of n
        temp=[]
        P(k,k,yb,[],temp)
        if n==k:
            for i in [1..len(temp)]:
                LL.append(list(add(L,temp[i-1])))
            return
        for i in [1..len(temp)]:
            PP(n-k,min(n-k,k),temp[i-1],list(add(L,temp[i-1])),[],LL)

def F(n):                                                             #This function gives the class of the punctual Hilbert
    LL=[];initial=[]                                                  #scheme of n points of A^3 for n<=5 with error terms
    for i in [1..n]:                                                  #coming from strata not being affine spaces
        initial.append(n)
    PP(n,n,initial,[],[],LL)
    print(LL)                                                         #it prints all the 2-dim partitions of n
    print(len(LL))                                                    #it prints the number of 2-dim partitions of n
    result=0;temp=0
    
    for j in [0..len(LL)-1]:
        hd=1;print(j)                                                 #it prints the progress
        t={(i,j,k):0 for i in [0..n] for j in [0..n] for k in [0..n]}
        transform(LL[j],t)
        if len(LL[j])==1:
            hd=0
        else:
            s=0
            for i in range(len(LL[j])):
                s+=LL[j][i][0]
            if s==n:
                hd=0
        temp=Dimension(t,n,hd)
        result+=x^temp
    print(result)

    
F(4)                                                                  #This is an example for n=4

[[[1], [1], [1], [1]], [[1, 1], [1], [1]], [[1, 1], [1, 1]], [[2], [1], [1]], [[2], [2]], [[1, 1, 1], [1]], [[2, 1], [1]], [[3], [1]], [[1, 1, 1, 1]], [[2, 1, 1]], [[2, 2]], [[3, 1]], [[4]]]
13
0
1
2
3
4
5
6


TypeError: no canonical coercion from Symbolic Ring to Rational Field

In [17]:
R.<l, t> = ZZ[]


def inverse(f):                                                       #This function calculate the first 6 terms of the
    a={(i):var("a_{}".format(i)) for i in [1..6]}                    #inverse of f in the ring 1+tR[[t]]
    b={(i):var("b_{}".format(i)) for i in [1..6]}
    for i in [1..6]:
        a[i]=f.coefficient({t:i})
    
    for i in [1..6]:
        temp=0
        for j in range(i-1):
            temp+=a[i-1-j]*b[j+1]
        b[i]=-a[i]-temp
    
    temp=1
    for i in [1..6]:
        temp+=b[i]*t^i
    
    return temp

def zeta(a,n):                                                       #This function calculate (1-t^n)^(-a) where
    Temp=1;temp=0                                                    #a is a polynomial in the Lefschetz class L
    if type(a)==type(0):
        for j in range(6):
            temp+=t^(n*j)
        return temp^a
    else:       
        c=a.degree(x)+1
        for i in range(c):
            temp=0
            for j in range(6):
                temp+=(l^(i*j))*(t^(n*j))
            if a.coefficients(sparse=False)[i]>=0:    
                Temp=Temp*(temp^(a.coefficients(sparse=False)[i]))
            else:
                Temp=Temp*(inverse(temp^(-a.coefficients(sparse=False)[i])))
        return Temp
    
    
def M(a,b):                                                        #This function keeps the first 6 terms of a*b
    temp=0;c=a*b
    for i in range(6):
        temp+=c.coefficient({t:i})*(t^i)
    return temp
                

R.<x>=ZZ[]                                       #We calculate [Hilb^n(A^3)] based on [Hilb^n_0(A^3)] via power structures
a=M(M(M(M(zeta(1,1),zeta(x^2+x,2)),zeta(x^4+x^3+x^2,3)),zeta(x^6+2*x^5+x^4+x^3-x^2,4)),zeta(x^8+2*x^7+2*x^6+x^5-x^3,5))
print("punctual Hilbert scheme of n points of A^3 for 0<=n<=5")
for i in range(6):
    print(a.coefficient({t:i}))
b=M(M(M(M(zeta(x^3,1),zeta(x^5+x^4,2)),zeta(x^7+x^6+x^5,3)),zeta(x^9+2*x^8+x^7+x^6-x^5,4)),zeta(x^(11)+2*x^(10)+2*x^9+x^8-x^6,5))
print("Hilbert scheme of n points of A^3 for 0<=n<=5")
for i in range(6):
    print(b.coefficient({t:i}))

punctual Hilbert scheme of n points of A^3 for 0<=n<=5
1
1
l^2 + l + 1
l^4 + l^3 + 2*l^2 + l + 1
l^6 + 2*l^5 + 3*l^4 + 3*l^3 + 2*l^2 + l + 1
l^8 + 2*l^7 + 4*l^6 + 5*l^5 + 5*l^4 + 3*l^3 + 2*l^2 + l + 1
Hilbert scheme of n points of A^3 for 0<=n<=5
1
l^3
l^6 + l^5 + l^4
l^9 + l^8 + 2*l^7 + l^6 + l^5
l^12 + l^11 + 3*l^10 + 3*l^9 + 4*l^8 + l^7 + l^6 - l^5
l^15 + l^14 + 3*l^13 + 4*l^12 + 7*l^11 + 5*l^10 + 4*l^9 - l^6
